Need opencv and mediapipe libraires<br>
To install:<br>
!pip install opencv-python<br>
!pip install mediapipe

In [3]:
!pip install import-ipynb

In [19]:
!pip install pycaw

     -------------------------------------- 184.3/184.3 kB 2.8 MB/s eta 0:00:00


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import import_ipynb
import hand_landmarks_module as hlm
import time
import math

from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

importing Jupyter notebook from hand landmarks module.ipynb


In [3]:
#Creating an object of a class
hand=hlm.handDetector()

In [4]:
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
print(volume.GetVolumeRange())


(-96.0, 0.0, 0.125)


In [5]:
minVol,maxVol=volume.GetVolumeRange()[0],volume.GetVolumeRange()[1]

In [44]:
cap=cv2.VideoCapture(0)

pTime=0

while True:
    ret,frame=cap.read()
    hand.findHand(frame)
    landmark_li=hand.findPosition(frame,draw=False)
    if len(landmark_li)!=0:
        #print(landmark_li)
        x1,y1=landmark_li[4][1],landmark_li[4][2]          #Thumb Tip x,y coordinates
        x2,y2=landmark_li[8][1],landmark_li[8][2]          #Index Tip x,y coordinates
        
        # Drawing circles on these two tips
        cv2.circle(frame,(x1,y1),12,(196, 255, 151),-1)
        cv2.circle(frame,(x2,y2),12,(196, 255, 151),-1)
        
        
        # Drawing a line between the these two points
        cv2.line(frame,(x1,y1),(x2,y2),(196, 255, 151),4)
        
        
        # Calculating the distance between these two which is later on used to adjust the volume 
        # Can also use math.hypot( x2-x1, y2-y1 )
        dist=math.sqrt(((x2-x1)**2)+(y2-y1)**2)                 
        
        
        # Distance range---> 30-180
        # Converting this range into the range which will be able to control the volume on our system
        # Setting the value in vol as the system volume
        vol=np.interp(dist,[30,180],[minVol,maxVol])                   # Returns single value
        volume.SetMasterVolumeLevel(vol, None)
        
        
        # To convert the vol variable to actual volume range(0-100)
        barVol=np.interp(vol,[minVol,maxVol],[0,100])
        
        # To get the height of the rectangle to be filled so that we can draw it as a bar
        bar=np.interp(vol,[minVol,maxVol],[260,90])
        
        #To create bar volume
        cv2.rectangle(frame,(90,260),(60,int(bar)),(255, 253, 115),-1)
        
        
    ''' After incr/decr the volume, the final value of barVol variable is still printed even after taking the hand away from the 
     webcam. Ex: If 45 was the last value in barVol, when the hand is moved away from the webcam it shows 45 instead
     of 0. So we do this:'''
    if len(landmark_li)==0:
        barVol=0
        
    # To print the outer rectangle of the bar
    cv2.rectangle(frame,(60,90),(90,260),(255, 253, 115),3)
    
    #To print the volume percentage
    cv2.putText(frame,str(int(barVol))+"%",(62,295),cv2.FONT_HERSHEY_COMPLEX,0.7,(255, 253, 115),2)
    
    # To print the fps(OPTIONAL)
    cTime=time.time()
    fps=1/(cTime-pTime)
    pTime=cTime
    cv2.putText(frame,'FPS: '+str(int(fps)),(40,50),cv2.FONT_HERSHEY_COMPLEX,0.6,(0,255,0),2)
    
    cv2.imshow('VOLUME CONTROLLER',frame)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
        
        
cap.release()
cv2.destroyAllWindows()

EXTRAS: Doesn't work with bluetooth connected